# Using Environ for Solvent and Electrolyte Effects on Slabs

## Tutorial Setup

Import all the Python modules needed in this tutorial. Most of these are very common in scientific computing, some are popular tools in atomistic simulations.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
#
# ASE is a very convenient module for setting up simulations on molecules and 
# bulk materials
#
from ase.build import molecule, fcc111, fcc100, add_adsorbate
from ase.visualize import view
from ase.calculators.espresso import Espresso

Setting up environment variables that are needed in order to submit simulations using Quantum Espresso through ASE

In [ ]:
os.environ['ASE_ESPRESSO_COMMAND'] = "mpirun -np 4 /Users/oliviero/PWSCF/espresso-git/bin/pw.x --environ -in PREFIX.pwi > PREFIX.pwo"
os.environ['OMP_NUM_THREADS'] = "1"

Some basic constants that may be useful later in the tutorials

In [ ]:
eV2Ry = 13.605662285137 # energy conversion factor
eV2kcal_mol = 23.0609 # energy conversion factor
bohr2ang = 0.5291772 # length conversion factor
ang2bohr = 1./bohr2ang

## Local Functions and Classes Used for the Tutorial

Basic functions to extract data from pw.x output files

In [ ]:
def get_total_energy(filename='espresso.pwo'):
    """
    Given a filename corresponding to a pw.x output, 
    extract the total energies from each scf calculation

    Input Variables:
        filename = name of pw.x output file
    
    Output Variables:
        energies = list of floats
    """
    energies=[]
    lines = [line for line in open(filename, 'r')]
    for line in lines:
        if line.strip().startswith('!'):
            energies.append(float(line.split()[4]))
    return energies

def get_scf_energy(filename='espresso.pwo'):
    """
    Given a filename corresponding to a pw.x output, 
    extract the energies from each scf step

    Input Variables:
        filename = name of pw.x output file
    
    Output Variables:
        energies = list of floats
    """
    energies=[]
    lines = [line for line in open(filename, 'r')]
    for line in lines:
        if line.strip().startswith('total energy'):
            energies.append(float(line.split()[3]))
    return energies

def get_Fermi_energy(filename='espresso.pwo'):
    """
    Given a filename corresponding to a pw.x output, 
    extract the Fermi energies from each scf cycle

    Input Variables:
        filename = name of pw.x output file
    
    Output Variables:
        Fermi energies = list of floats (in eV)
    """
    energies=[]
    lines = [line for line in open(filename, 'r')]
    for line in lines:
        if line.strip().startswith('the Fermi energy'):
            energies.append(float(line.split()[4]))
    return energies

def get_scf_accuracy(filename='espresso.pwo'):
    """
    Given a filename corresponding to a pw.x output, 
    extract the energies from each scf step

    Input Variables:
        filename = name of pw.x output file
    
    Output Variables:
        energies = list of floats
    """
    accuracies=[]
    lines = [line for line in open(filename, 'r')]
    for line in lines:
        if line.strip().startswith('estimated scf accuracy'):
            accuracies.append(float(line.split()[4]))
    return accuracies

def get_total_force(filename='espresso.pwo'):
    """
    Given a filename corresponding to a pw.x output, 
    extract the total force acting on the atoms from each scf calculation

    Input Variables:
        filename = name of pw.x output file
    
    Output Variables:
        forces = list of floats
    """
    forces=[]
    lines = [line for line in open(filename, 'r')]
    for line in lines:
        if line.strip().startswith('Total force'):
            forces.append(float(line.split()[3]))
    return forces

def get_bond_length(i,j,filename='espresso.pwo'):
    """
    Given a filename corresponding to a pw.x output, and the indexes of two
    atoms in the simulation (using Python notation, starting from 0)
    extract the bond length for each scf step in the simulation

    Input Variables:
        i = index of first atom
        j = index of second atom
        filename = name of pw.x output file
    
    Output Variables:
        bonds = list of floats
    """
    bonds=[]
    ri = np.zeros(3)
    rj = np.zeros(3)
    lines = [line for line in open(filename, 'r')]
    for line in lines:
        if line.strip().startswith('number of atoms'):
            nat = int(line.split()[4])
            break
    if i < 0 or i >= nat : 
        print('Error, index i must be >= 0 and < nat')
        return
    if j < 0 or j >= nat : 
        print('Error, index j must be >= 0 and < nat')
        return
    if i == j : 
        print('Error, indexes i and j must be different')
        return
    iat = -1
    for line in lines:
        if iat >= 0 and iat < nat : 
            if iat == i : ri = np.array([line.split()[1:4]],dtype=float)
            if iat == j : rj = np.array([line.split()[1:4]],dtype=float)
            iat += 1 
        if iat == nat : 
            bonds.append(np.sqrt(np.sum((ri-rj)**2)))
            iat = -1
        if line.strip().startswith('ATOMIC_POSITIONS'):
            iat = 0
    return bonds

Environ-specific function to retrieve the shift in energies and potential due to Gaussian-smeared nuclei

In [ ]:
def get_Gaussian_shift(filename='espresso.pwo'):
    """
    Given a filename corresponding to a pw.x output, 
    extract the potential shift due to the Gaussin-smeared nuclei 
    used by Environ

    Input Variables:
        filename = name of pw.x output file
    
    Output Variables:
        energy shift = list of floats (in eV)
    """
    shifts=[]
    lines = [line for line in open(filename, 'r')]
    for line in lines:
        if line.strip().startswith('the potential shift due'):
            shifts.append(float(line.split()[9]))
    return shifts

Cubefile visualization tools

In [ ]:
class cubefile():
    """
    Class to read and manipulate cubefiles
    """

    def __init__(this,filename,delta=1.e-5):
        lines = [line for line in open(filename, 'r')]
        this.nat = int(lines[2].split()[0])
        this.origin = np.array(lines[2].split()[1:4],dtype=float)
        this.n = np.zeros(3,dtype=int)
        this.dgrid = np.zeros(3)
        for i in range(3):
            this.n[i] = int(lines[3+i].split()[0])
            this.dgrid[i] = float(lines[3+i].split()[i+1])
        this.atmnum = np.zeros(this.nat,dtype=int)
        this.atmchg = np.zeros(this.nat)
        this.atmpos = np.zeros((3,this.nat))
        for i in range(this.nat):
            this.atmnum[i] = int(lines[6+i].split()[0])
            this.atmchg[i] = float(lines[6+i].split()[1])
            this.atmpos[:,i] = np.array(lines[6+i].split()[2:5],dtype=float)
        this.ntot = this.n[0]*this.n[1]*this.n[2]
        this.data = np.zeros(this.ntot) 
        i = 0
        for line in lines[6+this.nat:-1] : 
            this.data[i:i+6]=[ float(s) for s in line.split()]
            i += 6
        this.data[i:]=[ float(s) for s in line.split()]
        this.data = this.data.reshape((this.n[0],this.n[1],this.n[2]))
        this.r=np.zeros((3,this.n[0],this.n[1],this.n[2]))
        this.x=np.arange(0.,this.dgrid[0]*this.n[0]-delta,this.dgrid[0])+this.origin[0]
        this.y=np.arange(0.,this.dgrid[1]*this.n[1]-delta,this.dgrid[1])+this.origin[1]
        this.z=np.arange(0.,this.dgrid[2]*this.n[2]-delta,this.dgrid[2])+this.origin[2]
        this.r[0],this.r[1],this.r[2]=np.meshgrid(this.x,this.y,this.z,indexing='ij')

    def to_line(this,center,axis):
        icenter = np.rint(center/this.dgrid).astype('int')
        icenter = icenter - this.n * np.trunc(icenter//this.n).astype('int') 
        if axis == 0 :
            axis = this.r[0,:,icenter[1],icenter[2]]
            value = this.data[:,icenter[1],icenter[2]]
        elif axis == 1 :
            axis = this.r[1,icenter[0],:,icenter[2]]
            value = this.data[icenter[0],:,icenter[2]]
        elif axis == 2 :
            axis = this.r[2,icenter[0],icenter[1],:]
            value = this.data[icenter[0],icenter[1],:]
        return axis,value
    
    def to_line_planar_average(this,center,axis):
        icenter = np.rint(center/this.dgrid).astype('int')
        icenter = icenter - this.n * np.trunc(icenter//this.n).astype('int') 
        if axis == 0 :
            axis = this.r[0,:,icenter[1],icenter[2]]
            value = np.mean(this.data,(1,2))
        elif axis == 1 :
            axis = this.r[1,icenter[0],:,icenter[2]]
            value = np.mean(this.data,(0,2))
        elif axis == 2 :
            axis = this.r[2,icenter[0],icenter[1],:]
            value = np.mean(this.data,(0,1))
        return axis,value
    
    def to_surface(this,center,axis):
        icenter = np.rint(center/this.dgrid).astype('int')
        icenter = icenter - this.n * np.trunc(icenter//this.n).astype('int') 
        if axis == 0 :
            xx = this.r[1,icenter[0],:,:]
            yy = this.r[2,icenter[0],:,:]
            zz = this.data[icenter[0],:,:]
        elif axis == 1 :
            xx = this.r[0,:,icenter[1],:]
            yy = this.r[2,:,icenter[1],:]
            zz = this.data[:,icenter[1],:]
        elif axis == 2 :
            xx = this.r[0,:,:,icenter[2]]
            yy = this.r[1,:,:,icenter[2]]
            zz = this.data[:,:,icenter[2]]
        return xx,yy,zz

## Modeling a Pt (111) Surface with a CO Adsorbate

In [ ]:
slab = fcc111('Pt', size=(2,2,2), a=3.98, vacuum=7.5, orthogonal=True)
co = molecule('CO')
add_adsorbate(slab,co,3.6,'ontop')

In [ ]:
view(slab, viewer="x3d")

In [ ]:
pseudopotentials = {
    "C":"C.pbe-rrkjus.UPF",
    "O":"O.pbe-rrkjus.UPF",
    "Pt":"Pt.pbe-n-rrkjus_psl.0.1.UPF"
}

In [ ]:
slab.cell

## Neutral Slab in Vacuum

We can start with the simplest environment, which just takes care of fixing PBC artifacts. Also in this case we will use a parabolic correction of the potential. For neutral slabs, such a correction is sometimes konwn as the dipole correction (e.g., as introduced by L. Bengtsson, Phys. Rev. B 59, 12301).

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': '../pseudos',
        'calculation': 'scf',
        'prefix': 'PtCO'
    },
    'system': {
        'ecutwfc': 30,
        'ecutrho': 300,
        'occupations': 'smearing',
        'degauss': 0.03,
        'smearing': 'mv',
        'nbnd': 80,
        'tot_charge': 0.
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.2
    }
} 

calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

slab.calc = calc

Note that we need to specify that we are dealing with a 2D system and the direction of the axis perpendicular to the plane (most of the time it will be the third axis, a.k.a. z).

In [ ]:
%%bash
  cat > environ.in << EOF
&ENVIRON
   !
   verbose = 2
   environ_thr = 10 ! this controls when Environ kicks in
   environ_type = 'vacuum' ! this is the default value, no environment effects
   env_electrostatic = .true. ! this will still activate Environ for electrostatics
   !
/
&BOUNDARY
   !
   solvent_mode = 'full'
   !
/
&ELECTROSTATIC
   !
   pbc_correction = 'parabolic'
   pbc_dim = 2
   pbc_axis = 3
   !
/
EOF

In [ ]:
energy = slab.get_potential_energy()
print(f"Energy in vacuum = {energy:.3f} eV")

In the presence of the parabolic correction, the resulting potential will be aligned to zero. This implies that the Fermi energy of the system can be identified as the workfunction of the surface of the material. However, we need to make sure to include a constant shift reported by Environ in the pw.x output, related to the way Environ handles nuclear charges.  

In [ ]:
print("The Fermi energy (workfunction) of the system is {} eV".format(get_Fermi_energy()))
print("An additional shift of {:6.4f} eV in the energy and potential needs to be included".format(get_Gaussian_shift()[0]))

We can also look at the potentials produced by Environ and verify that the behaviors make physical sense.

In [ ]:
vref_neutral_vac=cubefile('vreference.cube')
vel_neutral_vac=cubefile('velectrostatic.cube')
dv_neutral_vac=cubefile('dvtot.cube')

In [ ]:
x,y=vref_neutral_vac.to_line_planar_average(slab.positions[-1,:]*ang2bohr,2)
plt.plot(x,y)
#x,y=vel_neutral_vac.to_line_planar_average(slab.positions[-1,:]*ang2bohr,2)
#plt.plot(x,y)
#x,y=dv_neutral_vac.to_line_planar_average(slab.positions[-1,:]*ang2bohr,2)
#plt.plot(x,y)
plt.ylabel('Planar-Averaged Electrostatic Potential (Ry)')
plt.xlabel('z (Bohr)')
plt.show()

## Charged Slab in Vacuum

While PBC artifacts already affect neutral slabs, if we want to study charged 2D systems we are in troubles.

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': '../pseudos',
        'calculation': 'scf',
        'prefix': 'PtCO'
    },
    'system': {
        'ecutwfc': 30,
        'ecutrho': 300,
        'occupations': 'smearing',
        'degauss': 0.03,
        'smearing': 'mv',
        'nbnd': 80,
        'tot_charge': 1.
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.2
    }
} 

calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

slab.calc = calc

In [ ]:
energy = slab.get_potential_energy()
print(f"Energy in vacuum = {energy:.3f} eV")

In [ ]:
vref_charged_vac=cubefile('vreference.cube')
vel_charged_vac=cubefile('velectrostatic.cube')
dv_charged_vac=cubefile('dvtot.cube')

In [ ]:
x,y=vref_charged_vac.to_line_planar_average(slab.positions[-1,:]*ang2bohr,2)
plt.plot(x,y)
#x,y=vel_charged_vac.to_line_planar_average(slab.positions[-1,:]*ang2bohr,2)
#plt.plot(x,y)
#x,y=dv_charged_vac.to_line_planar_average(slab.positions[-1,:]*ang2bohr,2)
#plt.plot(x,y)
plt.ylabel('Planar-Averaged Electrostatic Potential (Ry)')
plt.xlabel('z (Bohr)')
plt.show()

## Dielectric Effects

Could it be that including a screening dielectric medium solves the problem?

In [ ]:
%%bash
  cat > environ.in << EOF
&ENVIRON
   !
   verbose = 2
   environ_thr = 10 ! this controls when Environ kicks in
   env_static_permittivity = 78.3
   !
/
&BOUNDARY
   !
   solvent_mode = 'full'
   !
/
&ELECTROSTATIC
   !
   pbc_correction = 'parabolic'
   pbc_dim = 2
   pbc_axis = 3
   !
/
EOF

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': '../pseudos',
        'calculation': 'scf',
        'prefix': 'PtCO'
    },
    'system': {
        'ecutwfc': 30,
        'ecutrho': 300,
        'occupations': 'smearing',
        'degauss': 0.03,
        'smearing': 'mv',
        'nbnd': 80,
        'tot_charge': 1.
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.2
    }
} 

calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

slab.calc = calc

In [ ]:
energy = slab.get_potential_energy()
print(f"Energy in vacuum = {energy:.3f} eV")

Note that for this specific system, the SCF is not very well behaved in the presence of the dielectric. The simulation parameters are probably the cause of the slow convergence.

In [ ]:
accuracy = get_scf_accuracy()
plt.semilogy(accuracy)

In [ ]:
vref_neutral_diel=cubefile('vreference.cube')
vel_neutral_diel=cubefile('velectrostatic.cube')
dv_neutral_diel=cubefile('dvtot.cube')

In [ ]:
x,y=vref_neutral_diel.to_line_planar_average(slab.positions[-1,:]*ang2bohr,2)
plt.plot(x,y)
x,y=vel_neutral_diel.to_line_planar_average(slab.positions[-1,:]*ang2bohr,2)
plt.plot(x,y)
#x,y=dv_neutral_diel.to_line_planar_average(slab.positions[-1,:]*ang2bohr,2)
#plt.plot(x,y)
plt.ylabel('Planar-Averaged Electrostatic Potential (Ry)')
plt.xlabel('z (Bohr)')
plt.ylim(-0.1,0.1)
plt.show()

In [ ]:
xx,yy,zz=dv_neutral_diel.to_surface(slab.positions[-1,:],1)
plt.contourf(xx,yy,zz,levels=100)
plt.axis('equal')

## Modeling the Electrolyte Ions: Helmholtz Planes

A dielectric by itself will not fully fix the problem of simulating a charged slab. We need to include a countercharge in the surrouding medium that fully neutralizes our simulation cell. The easiest way is to follow the model of the Helmholtz layers, which should approximate the most stable solution for free charges at 0K. 

In [ ]:
%%bash
  cat > environ.in << EOF
&ENVIRON
   !
   verbose = 2
   environ_thr = 10 ! this controls when Environ kicks in
!   env_static_permittivity = 78.3
   env_external_charges = 2
   !
/
&BOUNDARY
   !
   solvent_mode = 'full'
   !
/
&ELECTROSTATIC
   !
   pbc_correction = 'parabolic'
   pbc_dim = 2
   pbc_axis = 3
   !
/
EXTERNAL_CHARGES (bohr)
-0.5 0. 0. 7. 0.5 2 3
-0.5 0. 0. 30. 0.5 2 3
EOF

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': '../pseudos',
        'calculation': 'scf',
        'prefix': 'PtCO'
    },
    'system': {
        'ecutwfc': 30,
        'ecutrho': 300,
        'occupations': 'smearing',
        'degauss': 0.03,
        'smearing': 'mv',
        'nbnd': 80,
        'tot_charge': 1.
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.2
    }
} 

calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

slab.calc = calc

In [ ]:
energy = slab.get_potential_energy()
print(f"Energy in vacuum = {energy:.3f} eV")

Now, the calculation of the workfunction will correspond to the potential drop on the system. This will clearly depend on the position we choose for the Helmholtz layers. 

In [ ]:
Ef = get_Fermi_energy()[0] + get_Gaussian_shift()[0]
print('The potential on the system is = {:5.3} eV'.format(-Ef/eV2Ry))

In [ ]:
vref_helm_vac=cubefile('vreference.cube')
vel_helm_vac=cubefile('velectrostatic.cube')
dv_helm_vac=cubefile('dvtot.cube')

In [ ]:
x,y=vref_helm_vac.to_line_planar_average(slab.positions[-1,:]*ang2bohr,2)
plt.plot(x,y)
x,y=vel_helm_vac.to_line_planar_average(slab.positions[-1,:]*ang2bohr,2)
plt.plot(x,y)
plt.plot(x,np.ones(len(y))*Ef/eV2Ry)
plt.ylabel('Planar-Averaged Electrostatic Potential (Ry)')
plt.xlabel('z (Bohr)')
#plt.ylim(-0.1,0.1)
plt.show()

## Modeling the Electrolyte Ions: Poisson-Boltzmann Problems

Less empirical definitions of the electrochemical diffuse layers can be introduced. In Environ we have a hierarchy of models of increasing accuracy (and cost). Linearized Poisson-Boltzmann can be solved relatively fast and converges smoothly. The analytical solution to a planar-averaged problem is implemented as a PBC correction scheme, dubbed `gcs` (for the Gouy-Chapman-Stern model of the diffuse layer on which the equations are based). A full non-linear Poisson-Boltzmann algorithm is also implemented, although costs and convergence are far from ideal. For this latter model, a size-modified approach, that accounts for the finite size of the electrolyte ions, is more physical and shows better convergence.

In [ ]:
slab = fcc100('Ag', size=(1,1,8), a=4.1485, vacuum=20, orthogonal=True)
pseudopotentials={ "Ag":"Ag.pbe-n-rrkjus_psl.1.0.0.UPF"}

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': '../pseudos',
        'calculation': 'scf',
        'prefix': 'Ag'
    },
    'system': {
        'ecutwfc': 30,
        'ecutrho': 300,
        'occupations': 'smearing',
        'degauss': 0.02,
        'smearing': 'mv',
        'tot_charge': 0.5
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.2
    }
} 

calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(2,2,1),
    koffset=(0,0,0))

slab.calc = calc

In [ ]:
%%bash
cat > environ.in << EOF
&ENVIRON
   verbose = 0
   environ_thr = 1.d2
   env_static_permittivity = 78.3
   env_electrolyte_ntyp = 2
   zion(1) = 1
   zion(2) = -1
   cion(1) = 1.0
   cion(2) = 1.0
   cionmax = 0
   temperature = 300
   system_dim = 2           
   system_axis = 3 
   system_ntyp = 1
   electrolyte_linearized = .true.
/
&BOUNDARY
   solvent_mode = 'full'
   electrolyte_mode = 'system'
   electrolyte_distance = 30.
   electrolyte_spread = 0.01   
/
&ELECTROSTATIC
   pbc_correction = 'parabolic'
   pbc_dim = 2
   pbc_axis = 3
   tol = 5.D-13
   inner_tol = 5.D-18
/
EOF

In [ ]:
energy = slab.get_potential_energy()
Ef = get_Fermi_energy()[0] + get_Gaussian_shift()[0]
print('The potential on the system is = {:5.3} eV'.format(-Ef/eV2Ry))

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': '../pseudos',
        'calculation': 'scf',
        'prefix': 'Ag'
    },
    'system': {
        'ecutwfc': 30,
        'ecutrho': 300,
        'occupations': 'smearing',
        'degauss': 0.02,
        'smearing': 'mv',
        'tot_charge': 0.5
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.2
    }
} 

calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(2,2,1),
    koffset=(0,0,0))

slab.calc = calc

In [ ]:
%%bash
cat > environ.in << EOF
&ENVIRON
   verbose = 0
   environ_thr = 1.d2
   env_static_permittivity = 78.3
   env_electrolyte_ntyp = 2
   zion(1) = 1
   zion(2) = -1
   cion(1) = 1.0
   cion(2) = 1.0
   cionmax = 0
   temperature = 300
   system_dim = 2           
   system_axis = 3 
   system_ntyp = 1
   electrolyte_linearized = .false.
/
&BOUNDARY
   solvent_mode = 'full'
   electrolyte_mode = 'system'
   electrolyte_distance = 30.
   electrolyte_spread = 0.01   
/
&ELECTROSTATIC
   pbc_correction = 'gcs'
   pbc_dim = 2
   pbc_axis = 3
   tol = 5.D-13
   inner_tol = 5.D-18
/
EOF

In [ ]:
energy = slab.get_potential_energy()
Ef = get_Fermi_energy()[0] + get_Gaussian_shift()[0]
print('The potential on the system is = {:5.3} eV'.format(-Ef/eV2Ry))

In [ ]:
%%bash
cat > environ.in << EOF
&ENVIRON
   verbose = 0
   environ_thr = 1.d2
   env_static_permittivity = 78.3
   env_electrolyte_ntyp = 2
   zion(1) = 1
   zion(2) = -1
   cion(1) = 1.0
   cion(2) = 1.0
   cionmax = 10
   temperature = 300
   system_dim = 2           
   system_axis = 3 
   system_ntyp = 1
   electrolyte_linearized = .false.
/
&BOUNDARY
   solvent_mode = 'full'
   electrolyte_mode = 'system'
   electrolyte_distance = 30.
   electrolyte_spread = 0.01   
/
&ELECTROSTATIC
   pbc_correction = 'gcs'
   pbc_dim = 2
   pbc_axis = 3
   tol = 5.D-13
   inner_tol = 5.D-18
/
EOF